In [1]:
include("kernel.jl");

In [2]:
"""
weight function
"""
function wcos(r)
    if (abs(r) >=1) 
        return 0.
    else
        return (1.0 + cos(π * r))/2.0
    end
end

wcos

In [3]:
"""
domain radius
"""
radius = 0.50

radius

In [4]:
"""
Jacobian function

Projection to surface
"""

Jacobian(p, η) = 1.0 - η/norm(p)
Projection(r, p) = r*p/norm(p)

Projection (generic function with 1 method)

In [5]:
"""
The kernel function
"""
function ϕ(target, source)
    v = target - source
    r = norm(v)
    surf_n = source/norm(source)
    κ = 1.0/norm(source)
    
    return Jacobian(source, norm(source) - radius)*(r > 1e-10 ? -dot(v, surf_n)/r^2/2π : κ/4π)
end

ϕ

In [6]:
# shifts
yshift = 0.
xshift = 0.

M = 160
ϵm = 0.1

x_range = linspace(-1. + xshift, 1. + xshift, M)
y_range = linspace(-1. + yshift, 1. + yshift, M)

# build meshgrid
xs, ys = reshape(x_range, M, 1) , reshape(y_range, 1, M)
xs, ys = (repmat(xs, 1, M), repmat(ys, M, 1))

# dxdy/ϵ
ΔxΔy_div_ϵ = (x_range[2] - x_range[1]) * (y_range[2] - y_range[1])/ϵm


# count interior points
N = 0
for p in zip(xs, ys)
    if abs(norm(collect(p)) - radius) < ϵm
        N+=1
    end
end

source = zeros(N, 2)
target = zeros(N, 2)
weight = zeros(N)


# assign source/target, weight
sI = 1
for p in zip(xs, ys)
    pv = collect(p)
    normp = norm(pv)
    if abs(normp - radius) < ϵm
        target[sI, :] = source[sI, :] = (radius/normp) * pv
        weight[sI] = wcos((normp - radius)/ϵm)*ΔxΔy_div_ϵ
        sI += 1
    end
end



In [7]:
# t: target
# s: source
# n: normal (not used)

function g(s,t)
    return ϕ(t,s)
end

g (generic function with 1 method)

In [8]:
bbfmm = BBFMM();
np = 5;

In [9]:
using KrylovMethods

In [10]:
function A(x)
    initialize!(bbfmm, np, source, target, x.*weight, N, N, 80, 10, g);
    return 0.5 * x + FMM!(bbfmm)
end

A (generic function with 1 method)

In [11]:
testCharge=ones(N)
@time y =  bicgstb(A, testCharge, tol=1e-10, maxIter=100 , x = rand(N), out=2);
"""
gmres take more time usually
"""
# @time y =  gmres(A, testCharge, 30 , x = rand(N), out=2); 

=== bicgstb ===
iter	 relres
  1	1.70e-02
  2	1.06e-11
bcgstb achieved desired tolerance at iteration 2. Residual norm is 1.06e-11.
 10.451144 seconds (160.50 M allocations: 3.892 GB, 4.23% gc time)


In [12]:
M = 10000
target = 0.9 * rand(M, 2) - 0.45
checkPotential = zeros(M)
initialize!(bbfmm, np, source, target, y[1].*weight, N, M, 80, 10, g);
@time for i = 1:M
    for j = 1:N
        checkPotential[i] += bbfmm.kernel(bbfmm.tree.sourceTree[j,:], target[i, :])* bbfmm.chargeTree[j]
    end
end
@time potential = FMM!(bbfmm)
@printf("L∞ error is %e\n", maximum(abs(potential - checkPotential)))

 55.136498 seconds (1.15 G allocations: 29.532 GB, 5.12% gc time)
 10.713601 seconds (159.54 M allocations: 3.712 GB, 4.93% gc time)
L∞ error is 3.825333e-04
